In [4]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [1]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = os.environ.get('OPENAI_API_KEY'),
    project= os.environ.get('OPENAI_PROJECT_ID'),
    organization= os.environ.get('OPENAI_ORGANIZATION_ID')
)

In [7]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the purpose of life?"}
    ],
    max_tokens=50
)


C:\Users\Aviv9\AppData\Local\Temp\ipykernel_4352\3131962351.py:9: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response.dict()


{'id': 'chatcmpl-AtwenBb5jogyfpMT7dWOiSRoeKnR1',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': "The purpose of life is a deeply philosophical question and can vary greatly depending on individual beliefs, cultural backgrounds, and personal experiences. Some common perspectives include:\n\n1. **Biological Perspective**: From a scientific standpoint, the purpose of life can be seen as survival and reproduction. This perspective is centered around the idea of passing on genes to the next generation.\n\n2. **Philosophical Perspective**: Different philosophical schools offer various interpretations. For example:\n   - Existentialists might argue that life has no inherent purpose, and it's up to each individual to create their own meaning.\n   - Utilitarians may suggest that the purpose of life is to maximize happiness and reduce suffering.\n\n3. **Religious Perspective**: Many religions provide their own explanations for the purpose of 

In [2]:
def get_system_message():
    return """# Overview
You are tasked with summarizing a Reddit post related to SentinelOne and its competitors. Analyze the title and post text to extract key information useful for the marketing team. Follow the instructions carefully and structure the output as a JSON object.

# Instructions:
## Customer Status: Determine whether the poster is a customer of SentinelOne.
Output as `is_customer`: `True` or `is_customer`: `False`.

## Text Summary: Provide a brief summary of the post in one or two sentences.
Focus on the main situation or issue discussed.

## Competitor: Identify SentinelOne's competitor mentioned in the post.
Not all posts will mention a competitor.
Posts may mention multiple companies but focus only on the primary competitor.
Output as "competitor": "<Competitor Name>". If none is mentioned, use "competitor": "".

## Advantages of SentinelOne: List any advantages of SentinelOne over the competitor mentioned in the post.
Use only phrases from the following list:
* "Security Features"
* "Security Performance"
* "User-Friendly Interface"
* "Ease of Deployment"
* "Scalability"
* "Cross-Platform Support"
* "Visibility"
* "Customer Support"
* "Integration"
* "Cost"

If none are explicitly mentioned, leave as "advantages": [].

## Disadvantages of SentinelOne: List any disadvantages of SentinelOne over the competitor mentioned in the post.
Use only phrases from the following list:
* "Security Features"
* "Security Performance"
* "User-Friendly Interface"
* "Ease of Deployment"
* "Scalability"
* "Cross-Platform Support"
* "Visibility"
* "Customer Support"
* "Integration"
* "Cost"

If none are explicitly mentioned, leave as "disadvantages": [].

## Post Sentiment: Determine whether the post is in favor of SentinelOne, the competitor, or neutral.
Output as "sentiment": "Positive", "sentiment": "Negative", or "sentiment": "Neutral".

# Example:
## Input
Title: "Ninja Forcing Us to Pay $20,000 for SentinelOne License"
Text: "I need to vent about a frustrating situation we're dealing with at work. My colleague recently tried to *test* SentinelOne, which we apparently \"purchased\" through Ninja. Somehow, this turned into a $20,000 charge! The kicker? In our country, only a CEO can legally sign off on purchases of this nature. My colleague certainly doesn't have that authority.\n\nWe reached out to Ninja to explain the situation, but they're insisting we pay up. This seems ridiculous given the circumstances. Has anyone else dealt with something like this?\n\nHonestly, it feels like we're being strong-armed into paying for something we never intended to buy in the first place. \n\nUpdate:\n\nQuick update on the situation: I spoke with a representative from Ninja, and they were very understanding. We clarified the misunderstanding, and they agreed to remove the claim. Ninja handled it professionally, and I appreciate how cool they've been about the whole thing.\n\nI also want to clarify that we share a lot of the blame here. Ninja has been very professional about handling the situation. I'm glad we were able to resolve this amicably.\n\nThe big takeaway here is that we probably should have escalated the issue to the right person sooner.\nLesson learned! Thanks to everyone who offered advice and support!"

## Output:
{
  "is_customer": "False",
  "summary": "The poster describes a misunderstanding where their company was charged $20,000 for a SentinelOne license through Ninja, which was resolved amicably after discussions.",
  "competitor": "",
  "advantages": ["Customer Support"],
  "disadvantages": ["Cost"],
  "sentiment": "Natural"
}

# Notes:
Be concise in your output, but ensure all key details are captured.
If multiple competitors are mentioned, list only the primary one.
For advantages/disadvantages, infer based on explicit comparisons or implied sentiments in the text."""

In [3]:
from sales_competitor_analysis.reddit_data_collector import RedditDataCollector

reddit_data_collector = RedditDataCollector("")
post_file_path = r"C:\repos\sales_competitor_analysis\posts\104xsu8.json"
post = reddit_data_collector._load_post(post_file_path)

2025-01-26 16:12:11,365 - sales_competitor_analysis.reddit_data_collector - INFO - Loading post from C:\repos\sales_competitor_analysis\posts\104xsu8.json
2025-01-26 16:12:11,366 - sales_competitor_analysis.reddit_data_collector - INFO - Loaded post.id='104xsu8' from C:\repos\sales_competitor_analysis\posts\104xsu8.json


In [ ]:
results_json_schema = {
  "name": "customer_feedback",
  "schema": {
    "type": "object",
    "properties": {
      "customer_status": {
        "type": "string",
        "description": "Indicates whether the customer is currently active, inactive, or has any particular status."
      },
      "summary": {
        "type": "string",
        "description": "A brief overview of the customer's experience or issue."
      },
      "competitor": {
        "type": "string",
        "description": "The name of the competitor being discussed or compared."
      },
      "advantages": {
        "type": "array",
        "description": "List of advantages noted by the customer regarding the service or product.",
        "items": {
          "type": "string"
        }
      },
      "disadvantages": {
        "type": "array",
        "description": "List of disadvantages noted by the customer regarding the service or product.",
        "items": {
          "type": "string"
        }
      },
      "sentiment": {
        "type": "string",
        "description": "The overall sentiment of the customer, could be positive, negative, or neutral."
      }
    },
    "required": [
      "customer_status",
      "summary",
      "competitor",
      "advantages",
      "disadvantages",
      "sentiment"
    ],
    "additionalProperties": false
  },
  "strict": true
}

In [4]:
def process_post_for_prompt(post):
    return f"""Title: "{post.title}"
Text: "{post.text}" """

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": get_system_message()},
        {"role": "user", "content": process_post_for_prompt(post)}
    ],
    max_tokens=1000,
    response_format={"type": "json_object"}
)

In [5]:
print(response.choices[0].message.content)

{
  "is_customer": "False",
  "summary": "The poster is curious about the high regard for SentinelOne, questioning if it's due to its performance or cost compared to older AV solutions like Webroot, while noting past failures of SentinelOne in catching lateral movements.",
  "competitor": "Webroot",
  "advantages": [],
  "disadvantages": ["Security Performance"],
  "sentiment": "Negative"
}


In [6]:
from functools import partial
from sales_competitor_analysis.data_models.summarized_post import SummarizedPost
import importlib

importlib.import_module("sales_competitor_analysis")

a = SummarizedPost.model_validate_json(response.choices[0].message.content)

In [7]:
a.save_to_json_file(r"C:\repos\sales_competitor_analysis\posts\104xsu8_summary.json")